In [ ]:
import polars as pl
import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
commission = 1 / 10000.0
USE_HOUR_DATA = 0

In [ ]:

ROLLING_NUM = 600
fit_window_time = pl.duration (days = ROLLING_NUM)

In [ ]:
import os
LOAD_FILE_NO_CALC = 1


if USE_HOUR_DATA:
    result_hour_path = "data/agg_data_hour_to_day_alpha101.parquet"
    result_hour_path = "data/agg_data_hour_to_day_all_factor.parquet"
else:
    result_hour_path = 'data/result_hour.parquet'
    result_hour_path = 'data/result_hour_alpha101.parquet'
    # result_hour_path = 'data/daily_data_agg_from_hour_sep29.parquet'
    result_hour_path = "data/day_alpha101_sep29.parquet"
    result_hour_path = 'data/all_data_1d_boris_converted.parquet'
    result_hour_path = 'data/all_data_1d_boris_converted_with_amihud_alpha101.parquet'
    # result_hour_path = 'data/rolling_factors.parquet'
    # result_hour_path = 'data/selected_factors_0916_0913.parquet'

INPUT_FILE_NAME = result_hour_path.split ('/')[-1].split('.')[0]
print (f'INPUT_FILE_NAME: {INPUT_FILE_NAME}')

if LOAD_FILE_NO_CALC:
    if os.path.exists(result_hour_path):
        base_result_hour = pl.read_parquet(result_hour_path)
    else:
        assert 0, 'miss file'
base_result_hour

In [ ]:
# previous_result_hour = pl.read_parquet('data/result_hour.parquet')
# previous_result_hour = previous_result_hour.select(['open_time', 'symbol', 'open', 'return_skew',  'amihud', 'return_auto_corr_1_spearman_lag2'])
# # Join previous_result_hour and result_hour on symbol and open_time columns
# # Verify the new columns are added
# print(selected_result_hour.columns)

# previous_result_hour

supplement_factor_file = []
# supplement_factor_file.append (('data/result_hour.parquet', ['open', 'return_skew', 'amihud', 'return_auto_corr_1_spearman_lag2']))
# supplement_factor_file.append (('data/selected_factors_0916_0913.parquet', ['rolling_count_rank_20', 'rolling_quote_volume_market_share_pct_ewm_std_6']))

result_hour = base_result_hour

for each_file, factor_list in supplement_factor_file:
    supplement_result_hour = pl.read_parquet(each_file)
    supplement_result_hour = supplement_result_hour.select(factor_list + ['open_time', 'symbol'])
    result_hour = result_hour.join(
        supplement_result_hour,
        on=['open_time', 'symbol'],
        how='left'
    )

# # 将 open 放在第一列
# new_col_list = result_hour.columns
# new_col_list = [x for x in new_col_list if x != 'open']
# new_col_list = ['open'] + new_col_list
# result_hour = result_hour.select (new_col_list)


result_hour


In [ ]:
FACTOR_COMBINATION_LIST = ["return_skew", "amihud", "return_auto_corr_1_spearman_lag2"]
# FACTOR_COMBINATION_LIST = ['alpha54', 'alpha55']
for i in [13, 15, 16, 30, 33, 34, 35, 36, 45, 50, 51, 54, 55, 64, 71, 74, 99]:
    # for i in [44, 36, 30]:
    FACTOR_COMBINATION_LIST.append(f"alpha{i}")

FACTOR_COMBINATION_LIST = ["amihud"]
FACTOR_COMBINATION_LIST.append("rolling_count_rank_20")
FACTOR_COMBINATION_LIST.append("rolling_quote_volume_market_share_pct_ewm_std_6")

FACTOR_COMBINATION_LIST = [
    "rolling_quote_volume_market_share_pct_var_6",
    "rolling_alpha24_rank_40",
    "rolling_taker_buy_ratio_sum_6",
    "rolling_amihud_sum_40",
    "rolling_alpha24_self_cov_lag1_6",
    "rolling_alpha64_self_corr_lag1_6",
    "rolling_alpha40_skew_20",
    "rolling_alpha40_quantile_50_6",
    "rolling_alpha50_mean_20",
    "rolling_return_skew_6",
    "rolling_alpha64_self_corr_lag2_20",
    "rolling_taker_buy_ratio_zscore_40",
    "rolling_alpha46_self_cov_lag2_40",
    "rolling_alpha51_self_cov_lag2_6",
    "rolling_alpha30_zscore_6",
    "rolling_alpha55_self_cov_lag2_6",
    "rolling_alpha71_var_6",
    "rolling_alpha51_quantile_50_40",
    "rolling_alpha15_self_corr_lag2_20",
    "rolling_alpha54_ewm_var_40",
    "rolling_high_low_ratio_self_corr_lag2_20",
    "rolling_alpha54_quantile_50_40",
    "rolling_alpha45_var_40",
    "rolling_alpha24_mean_40",
    "rolling_alpha36_sum_6",
    "rolling_alpha54_rank_6",
    "rolling_alpha81_rank_6",
    "rolling_liquidity_ratio_self_corr_lag1_6",
    "rolling_alpha64_skew_6",
    "rolling_count_self_cov_lag2_40",
]


FACTOR_COMBINATION_LIST = ['amihud']
for i in [13, 15, 16, 30, 33, 34, 35, 36, 45, 50, 51, 54, 55, 64, 71, 74, 99]:
    FACTOR_COMBINATION_LIST.append(f"alpha{i}")

FACTOR_COMBINATION_LIST = ["return_skew", "amihud", "return_auto_corr_1_spearman_lag2"]
FACTOR_COMBINATION_LIST = [ "amihud", 'alpha30', 'alpha36', 'alpha40'] # this good
FACTOR_COMBINATION_LIST = [  'alpha30', 'alpha36', 'alpha45', 'alpha50']
FACTOR_COMBINATION_LIST = [ "amihud", 'alpha30', 'alpha36', 'alpha40'] # this good
FACTOR_COMBINATION_LIST = [ "amihud", 'alpha30', 'alpha36', 'alpha40', 'alpha45', 'alpha50']
# FACTOR_COMBINATION_LIST = ['amihud']
print(f"Factor Combination List: {FACTOR_COMBINATION_LIST}")

date_threshold = pl.datetime(2023, 1, 4)
origin_xgb_x_eval = result_hour.filter(pl.col("open_time") >= date_threshold).select(
    ["open_time", "symbol", "open", "close"] + FACTOR_COMBINATION_LIST
)
origin_xgb_x_eval

In [ ]:

# add future_ret in result_hour
UPDATE_POSITION_TIME = 7 # in day

if USE_HOUR_DATA:
    for i in range(1, UPDATE_POSITION_TIME + 1):
        result_hour = result_hour.with_columns(
            ((pl.col("open").shift(-i * 24) / pl.col("open") - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"future_{i}day_return")
        )
        # ).fill_null (0)
else:
    for i in range(1, UPDATE_POSITION_TIME + 1):
        result_hour = result_hour.with_columns(
            # ((pl.col("close").shift(-i) / pl.col("close") - 1) * 100)
            ((pl.col("open").shift(-i) / pl.col("open") - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"future_{i}day_return")
        )
        # ).fill_null (0)   

result_hour

## 因子组合
### Fama-Macbeth 线性组合

In [ ]:
import statsmodels.api as sm
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce

def count_null_inf_polars(df: pl.DataFrame):
    # Count null values
    null_counts = df.null_count().sum()
    
    # Count infinity values
    # inf_counts = df.select(
    #     pl.sum(pl.col('*').is_infinite().cast(pl.Int64))
    # ).item()
    inf_counts = 0
    
    return null_counts, inf_counts

def fama_macbeth_get_factor_weight (x_train, y_train, update_pos_days, symbol_num, factor_num):
    factor_weight_sum = np.zeros((symbol_num, factor_num))

    null_count, inf_count = count_null_inf_polars(x_train)
    # print(f"x_train contains {null_count} null values and {inf_count} infinity values")

    # Drop rows containing any null values
    x_train = x_train.drop_nulls()


    # result_hour = result_hour.sort(["symbol", "open_time"])
    total_weights_sum = np.zeros(factor_num)
    # unique_times = result_hour.select

    unique_times = x_train.select (pl.col("open_time").sort()).unique().to_numpy()

    constant_sum = 0.0

    for each_time in unique_times:
        slice_data = x_train.filter(pl.col("open_time") == each_time).fill_nan(0)
        X = slice_data[FACTOR_COMBINATION_LIST].to_numpy()
        X = sm.add_constant(X)  # 添加常数项（截距项）
        # y = slice_data[f"future_{UPDATE_POSITION_TIME}day_return"].to_numpy()
        # y = y_train[f"future_{update_pos_days}day_return"].to_numpy()
        y = slice_data[f"future_{update_pos_days}day_return"].to_numpy()
        # print (f' fit size: {X.shape} == {y.shape}')
        
        # print (each_time, X, y, X.shape, y.shape)

        model = sm.OLS(y, X)
        results = model.fit()
        weights = results.params[1:]
        constant_sum += results.params[0] # constant term
        # print (total_weights_sum.shape, weights.shape)

        while weights.shape[0] < total_weights_sum.shape[0]: # TODO, debug here, should not use this
            weights = np.append(weights, 0)
            # print (each_time)

        total_weights_sum += weights
    # print ('before divide ', total_weights_sum, len (unique_times))
    total_weights_sum /= len (unique_times)
    avg_const_term = constant_sum / len (unique_times)
    total_weights_sum

    # for i, weight in enumerate(total_weights_sum):
    #     print (f'{i} === {FACTOR_COMBINATION_LIST[i]} ==== {weight:.4f}')

    # 计算因子相关性矩阵
    factor_data = result_hour[FACTOR_COMBINATION_LIST].fill_nan(0)
    # print (factor_data)
    correlation_matrix = factor_data.corr()
    # print (correlation_matrix)

    # 绘制相关性矩阵的热图
    if 0:
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
        plt.title(f'Correlation Matrix between Factors for {update_pos_days}day')
        plt.show()

    # print (f'total weights sum {total_weights_sum}')
    weighted_factors = [x_train[col] * weight for col, weight in zip(FACTOR_COMBINATION_LIST, total_weights_sum)]
    # print (f'weight factors: {weighted_factors}')
    # return weighted_factors
    return total_weights_sum, avg_const_term

In [ ]:
from datetime import datetime

factor_num = len(FACTOR_COMBINATION_LIST)
# symbol_num = len(ret.columns) - 1
symbol_num = result_hour["symbol"].n_unique()

final_result_hour = result_hour.clone()

for cur_update_position_time in range(6, 8):
    # for cur_update_position_time in range(1, UPDATE_POSITION_TIME + 1):
    # for cur_update_position_time in range(UPDATE_POSITION_TIME, UPDATE_POSITION_TIME + 1):
    # prepare input
    non_nan_result_hour = result_hour.filter(
        pl.col(f"future_{cur_update_position_time}day_return").is_not_nan()
    )
    non_nan_result_hour = non_nan_result_hour.sort(["open_time", "symbol"])

    non_nan_linear_x = non_nan_result_hour.select(
        ["open_time", "symbol", f"future_{cur_update_position_time}day_return"]
        + FACTOR_COMBINATION_LIST
    )
    non_nan_linear_y = non_nan_result_hour.select(
        f"future_{cur_update_position_time}day_return"
    )

    # print ('non_nan_linear_x: ', non_nan_linear_x)
    # print (f'min date: {non_nan_linear_x["open_time"].min()} == max date: {non_nan_linear_x["open_time"].max()}')

    # Get all values in non_nan_linear_x['open_time'] as a list
    all_time_list = non_nan_linear_x["open_time"].to_list()
    cur_fit_start_date = date_threshold
    # fit_window_time = datetime.timedelta(days=14)
    max_date_range = non_nan_linear_x["open_time"].max()

    # print ('start : ', cur_fit_start_date, type (cur_fit_start_date))
    # print (f'max date range: {max_date_range} {type (max_date_range)}')

    cur_compound_factor_df = pl.DataFrame()

    # start rolling fit
    while 1:
        # while cur_fit_start_date.to_python() < max_date_range:
        # cur_fit_end_date = min (cur_fit_start_date + fit_window_time, max_date_range)
        cur_fit_end_date = cur_fit_start_date + fit_window_time
        # print (f'fit ======= {cur_fit_start_date} == {cur_fit_end_date}')

        linear_x_train = non_nan_linear_x.filter(
            pl.col("open_time") < cur_fit_start_date
        )
        # linear_x_eval = non_nan_linear_x.filter(pl.col('open_time') >= cur_fit_start_date)
        # cur_x_eval = origin_xgb_x_eval.filter ((pl.col ('open_time') >= cur_fit_start_date) & (pl.col ('open_time') < cur_fit_start_date + fit_window_time))
        cur_x_eval = origin_xgb_x_eval.filter(
            (pl.col("open_time") >= cur_fit_start_date)
            & (pl.col("open_time") < cur_fit_end_date)
        )

        train_size = linear_x_train.height
        # eval_size = linear_x_eval.height
        eval_size = cur_x_eval.height
        ratio = (
            train_size / eval_size if eval_size > 0 else float("inf")
        )  # Avoid division by zero

        linear_y_train = non_nan_linear_y.head(train_size)
        linear_y_eval = non_nan_linear_y.tail(non_nan_linear_y.height - train_size)

        # linear_x_train = linear_x_train.drop (['open_time', 'symbol'])

        # print (linear_x_train)
        # print ('linear y eval', linear_y_eval)
        # print("Training set size:", train_size)
        # print("Evaluation set size:", eval_size)
        # print("Ratio (Train:Eval):", ratio)

        weighted_factors, const_term = fama_macbeth_get_factor_weight(
            linear_x_train,
            linear_y_train,
            cur_update_position_time,
            symbol_num=symbol_num,
            factor_num=factor_num,
        )

        # print ('returned weight factor', weighted_factors, weighted_factors.shape, type (weighted_factors))

        weighted_sum_expr = pl.lit(const_term)  # Start with the const term, then sum all factors
        for factor, weight in zip(FACTOR_COMBINATION_LIST, weighted_factors):
            weighted_sum_expr += pl.col(factor) * weight

        # Create the 'compound_factor' column by applying the expression to the DataFrame
        # origin_xgb_x_eval = origin_xgb_x_eval.with_columns(weighted_sum_expr.alias(f'linear_compound_factor_{cur_update_position_time}day'))

        # print (f'cur x eval shape: {cur_x_eval.shape}')
        if cur_x_eval.shape[0] == 0:
            break

        cur_col_name = f"linear_compound_factor_{cur_update_position_time}day"
        cur_x_eval = cur_x_eval.with_columns(weighted_sum_expr.alias(cur_col_name))

        final_result_hour = final_result_hour.with_columns(
            pl.when(pl.col(f"future_{cur_update_position_time}day_return").is_null())
            .then(pl.lit(None))
            .otherwise(weighted_sum_expr)
            .alias(cur_col_name)
        )

        cur_x_eval = cur_x_eval.sort("open_time")
        cur_x_eval = cur_x_eval.select(pl.col(["open_time", "symbol", cur_col_name]))

        if cur_compound_factor_df.is_empty():
            cur_compound_factor_df = cur_x_eval
        else:
            assert cur_compound_factor_df.columns == cur_x_eval.columns
            cur_compound_factor_df = pl.concat([cur_compound_factor_df, cur_x_eval])

        # print (f'cur x eval {cur_fit_start_date} == {cur_fit_end_date}  === . {cur_x_eval} ===== after concat: {cur_compound_factor_df}')
        cur_fit_start_date = cur_fit_end_date

    # Left join cur_compound_factor_df on origin_xgb_x_eval based on open_time
    origin_xgb_x_eval = origin_xgb_x_eval.join(
        cur_compound_factor_df, on=["open_time", "symbol"], how="left"
    )

    # 将 compound_factor 添加为新的一列
    # origin_xgb_x_eval = origin_xgb_x_eval.with_column(compound_factor.alias(f'linear_compound_factor_{cur_update_position_time}day'))

    # print(origin_xgb_x_eval)

    # weighted_factors = [result_hour[col] * weight for col, weight in zip(FACTOR_COMBINATION_LIST, total_weights_sum)]

    # To sum these expressions, you can't use pl.sum directly on the list. Instead, sum them using a manual reduce:
    # from functools import reduce
    # predict_factor_expr = reduce(lambda a, b: a + b, weighted_factors)

    # # Assign the new column to the DataFrame
    # result_hour = result_hour.with_columns(predict_factor_expr.alias(f'linear_compound_factor_{cur_update_position_time}day'))
    # print (cur_update_position_time, result_hour)
    # break
# result_hour
# origin_xgb_x_eval
final_result_hour

In [ ]:

if USE_HOUR_DATA:
    # origin_xgb_x_eval.write_parquet('data/linear_compound_factor_hour_data.parquet')
    path = 'data/linear_compound_factor_hour_data.parquet'
    zero_clock_origin_xgb_x_eval = origin_xgb_x_eval.filter(pl.col("open_time").dt.hour() == 0)
    zero_clock_origin_xgb_x_eval.write_parquet('data/linear_compound_factor_hour_zero_clock.parquet')
else:
    path = f'data/linear_compound_factor_rolling_{ROLLING_NUM}_input_{INPUT_FILE_NAME}.parquet'
final_result_hour.write_parquet(path)
print (f'write into {path}')

final_result_hour

In [ ]:
final_result_hour.filter (pl.col ('symbol') == 'NOTUSDT').sort ('open_time')

In [ ]:
cur_symbol = 'DEFIUSDT'
# final_result_hour = origin_xgb_x_eval.clone()
final_result_hour.filter(pl.col('symbol') == cur_symbol).sort('open_time').filter (pl.col ('linear_compound_factor_7day').is_not_null())
final_result_hour.filter(pl.col('symbol') == cur_symbol).sort('open_time').filter (pl.col ('future_3day_return').is_not_null()).select (
    pl.col (['open_time', 'open', 'future_3day_return', 'future_7day_return','linear_compound_factor_7day'])
)

In [ ]:
final_result_hour.filter (pl.col ('symbol') == 'HNTUSDT').sort ('open_time')